In [ ]:
#importing the dataset
import csv
with open('transcripts.csv', encoding='latin-1') as TED_transcript:
    transcript_csv = csv.reader(TED_transcript)
    transcripts = list(transcript_csv)
    
#Converting the csv file to a list of strings (get rid of the url) and eliminating the second column
transcript_data = [t[0] for t in transcripts]
#Sanity check
#print(type(transcript_data))

In [3]:
#Preprocessing the data
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
regex = re.compile(r'[^\w\s]*') #regular expression to get rid of punctuation marks

def preprocessing(text):
    """
    Preprocesses text in a list of strings (removes punctuation, lowers words, tokenizes it and removes stop words from the list)

    Args:
        text: every string in the list of strings for the csv file

    Returns:
        processed_text: the text after preprocessing
    """
    word_tokens = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in word_tokens if word.lower() not in stop_words]
    processed_text = filtered_words
    processed_text = re.sub(regex,'', str(filtered_words))
    processed_text = processed_text.lower()
    return processed_text

for line in transcript_data:
    preprocessing(line)

In [ ]:
#Corpus creation


In [ ]:
#Word2Vec

In [ ]:
#PPMI

In [ ]:
#TF-IDF

In [ ]:
#Select word pairs

In [ ]:
#Cosine similarity

In [ ]:
#Heatmap Analysis
import sklearn

Analysis from Heatmap